In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Layer, LSTM, Dense, Flatten, Dropout, Conv1D, InputLayer, BatchNormalization, SimpleRNN
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# Step 1: Generate Training Data
def generate_strings(num_samples):
    data = []
    labels = []

    for _ in range(num_samples):
        # Generate strings that belong to the grammar {0+,1}
        string_length = np.random.randint(1, 61)  # Random string length up to 60 characters
        string = ['0' if np.random.rand() < 0.9 else '1' for _ in range(string_length)]
        
        # Pad or truncate the string to ensure it has a fixed length (e.g., 60)
        if len(string) < 60:
            string += ['0'] * (60 - len(string))
        elif len(string) > 60:
            string = string[:60]
        
        # # Pad the string to a length of 80
        string += ['0'] * (80 - len(string))
        
        data.append(''.join(string))
        labels.append('1')  # 1 for strings that belong to the grammar

        # Generate counterexamples (strings that do not belong to the grammar)
        string_length = np.random.randint(1, 61)  # Random string length up to 60 characters
        string = ['1' if np.random.rand() < 0.9 else '0' for _ in range(string_length)]
        
        # Pad or truncate the string to ensure it has a fixed length (e.g., 60)
        if len(string) < 60:
            string += ['0'] * (60 - len(string))
        elif len(string) > 60:
            string = string[:60]
        
        # # Pad the string to a length of 80
        string += ['0'] * (80 - len(string))
        
        data.append(''.join(string))
        labels.append('0')  # 0 for counterexamples
    return data, labels

In [3]:
num_samples = 500
data, labels = generate_strings(num_samples)
len(data), len(labels)

(1000, 1000)

In [4]:
# Define a function to one-hot encode a string
def one_hot_encode(string, num_classes):
    return tf.keras.utils.to_categorical([int(c) for c in string], num_classes=num_classes)

num_classes = 2  # 0 or 1

# One-hot encode the strings
encoded_data = np.array([one_hot_encode(s, num_classes) for s in data])

# Split the data into training and testing sets
x_train, x_val, y_train, y_val = train_test_split(encoded_data, labels, test_size=0.2, random_state=42)

# Ensure that labels are integers (0 or 1) and convert them to NumPy arrays
y_train = np.array([int(label) for label in y_train])
y_val = np.array([int(label) for label in y_val])

In [5]:
def build_tdnn():
    num_classes = 2
    model = Sequential()
    model.add(InputLayer(input_shape=(80, num_classes)))
    model.add(Conv1D(1, 20, activation='relu'))
    # model.add(Conv1D(64, 20, activation='relu'))
    # model.add(BatchNormalization())
    model.add(Flatten())
    # model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))
    # model.add(Dropout(best_dropout_rate))  # Adding dropout layer to reduce overfitting
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

model_tdnn = build_tdnn()
model_tdnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 61, 1)             41        
                                                                 
 flatten (Flatten)           (None, 61)                0         
                                                                 
 dense (Dense)               (None, 1)                 62        
                                                                 
Total params: 103
Trainable params: 103
Non-trainable params: 0
_________________________________________________________________


In [6]:
def build_rnn():
    num_classes = 2
    model = Sequential()
    model.add(InputLayer(input_shape=(80, num_classes)))
    model.add(SimpleRNN(4, return_sequences=False, unroll=True))
    # model.add(SimpleRNN(128, activation='relu', return_sequences=True))
    # model.add(SimpleRNN(64, activation='relu', return_sequences=True))
    # model.add(SimpleRNN(64))
    # model.add(LSTM(128, activation='relu', return_sequences=True))
    # model.add(LSTM(64, activation='relu'))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

model_rnn = build_rnn()
model_rnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 4)                 28        
                                                                 
 dense_1 (Dense)             (None, 20)                100       
                                                                 
 dense_2 (Dense)             (None, 1)                 21        
                                                                 
Total params: 149
Trainable params: 149
Non-trainable params: 0
_________________________________________________________________


In [9]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train and evaluate the models
print("TDNN Evaluation")
# model_tdnn.fit(x_train, y_train, epochs=10, batch_size=32)
# model_tdnn.fit(x_train, y_train, epochs=25, batch_size=32, validation_data=(x_val, y_val), callbacks=[early_stopping])
model_tdnn.fit(x_train, y_train, epochs=25, batch_size=32, validation_data=(x_val, y_val))
tdnn_loss, tdnn_accuracy = model_tdnn.evaluate(x_val, y_val)
print("\nRNN Evaluation")
# model_rnn.fit(x_train, y_train, epochs=30, batch_size=32, validation_data=(x_val, y_val), callbacks=[early_stopping])
# model_rnn.fit(x_train, y_train, epochs=30, batch_size=16, validation_data=(x_val, y_val), callbacks=[early_stopping])
model_rnn.fit(x_train, y_train, epochs=30, batch_size=48, validation_data=(x_val, y_val))
rnn_loss, rnn_accuracy = model_rnn.evaluate(x_val, y_val)

print(f"\nTDNN - Val Loss: {tdnn_loss}, Val Accuracy: {tdnn_accuracy}")
print(f"RNN - Val Loss: {rnn_loss}, Val Accuracy: {rnn_accuracy}")

TDNN Evaluation
Epoch 1/25
25/25 [==============================] - 5s 22ms/step - loss: 0.8162 - accuracy: 0.5000 - val_loss: 0.7375 - val_accuracy: 0.5300
Epoch 2/25
25/25 [==============================] - 0s 12ms/step - loss: 0.7134 - accuracy: 0.5175 - val_loss: 0.6969 - val_accuracy: 0.5650
Epoch 3/25
25/25 [==============================] - 0s 10ms/step - loss: 0.6914 - accuracy: 0.5163 - val_loss: 0.6752 - val_accuracy: 0.5000
Epoch 4/25
25/25 [==============================] - 0s 9ms/step - loss: 0.6650 - accuracy: 0.5487 - val_loss: 0.6357 - val_accuracy: 0.6100
Epoch 5/25
25/25 [==============================] - 0s 9ms/step - loss: 0.6121 - accuracy: 0.6050 - val_loss: 0.5563 - val_accuracy: 0.6600
Epoch 6/25
25/25 [==============================] - 0s 8ms/step - loss: 0.5381 - accuracy: 0.6737 - val_loss: 0.4839 - val_accuracy: 0.7700
Epoch 7/25
25/25 [==============================] - 0s 9ms/step - loss: 0.4862 - accuracy: 0.7625 - val_loss: 0.4464 - val_accuracy: 0.8250
E

# Testing

In [10]:
# Load the data from the text file
data = np.loadtxt('test.txt', dtype=str)
# print("Before:", data)

new_data = []
labels = []
for line in data:
    line = line.replace(",", "")  # Remove commas
    data = line[:-1] # Drop the last element in each line
    label = line[-1] # Last element is class label
    new_data.append(data)
    labels.append(label)

# print(len(new_data))
# print(len(labels[0]))
# type(labels[0])

In [11]:
# Define a function to one-hot encode a string
def one_hot_encode(string, num_classes):
    return tf.keras.utils.to_categorical([int(c) for c in string], num_classes=num_classes)

num_classes = 2  # 0 or 1

# One-hot encode the strings
encoded_new_data = np.array([one_hot_encode(s, num_classes) for s in new_data])
# encoded_new_data = encoded_new_data.reshape(-1, 60, 2)
# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(encoded_new_data, labels, test_size=0.2, random_state=42)

In [12]:
# # TDNN Model
# tdnn_model = Sequential([
#     InputLayer(input_shape=(80, num_classes)),  # Input layer for one-hot encoded strings
#     Conv1D(64, 5, activation='relu'),  # 5 is the delay line size
#     Flatten(),
#     Dense(128, activation='relu'),  # Vary the number of neurons as needed
#     Dense(1, activation='sigmoid')  # Output layer with one neuron for binary classification
# ])

# # Compile the TDNN model
# tdnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# # RNN Model
# rnn_model = Sequential([
#     InputLayer(input_shape=(80, num_classes)),  # Input layer for one-hot encoded strings
#     SimpleRNN(64),  # Vary the number of neurons as needed
#     Dense(1, activation='sigmoid')  # Output layer with one neuron for binary classification
# ])

# # Compile the RNN model
# rnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Ensure that labels are integers (0 or 1) and convert them to NumPy arrays
y_test = np.array([int(label) for label in labels])
# y_test = np.array([int(label) for label in y_test])

# # Train the models
# model_tdnn.fit(X_train, y_train, epochs=10, batch_size=32)
# model_rnn.fit(X_train, y_train, epochs=10, batch_size=32)

# Evaluate the models on the test data
tdnn_loss, tdnn_accuracy = model_tdnn.evaluate(encoded_new_data, y_test)
rnn_loss, rnn_accuracy = model_rnn.evaluate(encoded_new_data, y_test)

# # Train the models
# tdnn_model.fit(X_train, y_train, epochs=10, batch_size=32)
# rnn_model.fit(X_train, y_train, epochs=10, batch_size=32)

# # Evaluate the models on the test data
# tdnn_loss, tdnn_accuracy = tdnn_model.evaluate(X_test, y_test)
# rnn_loss, rnn_accuracy = rnn_model.evaluate(X_test, y_test)

print("With Evaluate Method")
print("---------------------------------------------------------------")
print(f"TDNN - Test Loss: {tdnn_loss}, Test Accuracy: {tdnn_accuracy}")
print(f"RNN - Test Loss: {rnn_loss}, Test Accuracy: {rnn_accuracy}\n")

16/16 [==============================] - 0s 15ms/step - loss: 0.6878 - accuracy: 0.7280
With Evaluate Method
---------------------------------------------------------------
TDNN - Test Loss: 0.3660324215888977, Test Accuracy: 0.9240000247955322
RNN - Test Loss: 0.6877744793891907, Test Accuracy: 0.7279999852180481



In [13]:
# Make predictions on the test data
tdnn_predictions = model_tdnn.predict(encoded_new_data)
rnn_predictions = model_rnn.predict(encoded_new_data)

# Convert model outputs to binary predictions
threshold = 0.5  # Adjust the threshold as needed
tdnn_binary_predictions = (tdnn_predictions > threshold).astype(int)
rnn_binary_predictions = (rnn_predictions > threshold).astype(int)

# Calculate confusion matrices
tdnn_confusion_matrix = confusion_matrix(y_test, tdnn_binary_predictions)
rnn_confusion_matrix = confusion_matrix(y_test, rnn_binary_predictions)

print("TDNN Accuracy:", accuracy_score(y_test, tdnn_binary_predictions))
print("RNN Accuracy:", accuracy_score(y_test, rnn_binary_predictions))
# Display the confusion matrices
print("TDNN Confusion Matrix:")
print(tdnn_confusion_matrix)

print("\nRNN Confusion Matrix:")
print(rnn_confusion_matrix)

16/16 [==============================] - 2s 15ms/step
TDNN Accuracy: 0.924
RNN Accuracy: 0.728
TDNN Confusion Matrix:
[[223  27]
 [ 11 239]]

RNN Confusion Matrix:
[[119 131]
 [  5 245]]
